In [ ]:
import sys
import pathlib
sys.path.append(str(pathlib.PurePath(pathlib.Path.cwd().parent)))

import numpy as np
import pickle
import matplotlib.cm as cm
import matplotlib.axes as am
import matplotlib.pyplot as plt
%matplotlib widget

import dimod
import dwave
import dwave.system
from dwave.system import DWaveSampler, EmbeddingComposite, FixedEmbeddingComposite
import dwave.inspector
import dwave_networkx as dnx
import minorminer

from src.particle_funcs import distance_matrix as distance_matrix
from src.particle_funcs import io as particles_io
import src.leap_funcs.qubo.q_matrix as q_matrix

from src import leap_funcs as leap_funcs
from src.leap_funcs import embedding_quality
from src.leap_funcs.qubo import parameterstudy

from src import h5py_funcs
from src.h5py_funcs import inspections, discoveries, init_custom_getstates, io, parameterstudy_using_info_file

In [ ]:
study_name = 'sub_2'
info_file_name = r'study_params_small.h5'
study_name = 'sub_3'
study_name = 'sub_4'
study_name = 'sub_5'
study_name = 'sub_5_1_1'
study_name = 'sub_5_2'
study_name = 'sub_5_3'
#study_name = 'sub_5_3_2'

study_subs = ['', '_2']
study_subs = ['']

info_name = r'study_params_sub5_3'

info_file_name = [info_name + sub + '.h5' for sub in study_subs]


study_folder_path = [pathlib.Path.cwd().joinpath('01_out\\'+study_name+sub) for sub in study_subs]

info_file_name_path = [pathlib.Path.joinpath(sfp, ifn) for sfp, ifn in zip(study_folder_path, info_file_name)]
samples_folder_name = r'samples'
samples_folder_name_path = [pathlib.Path.joinpath(sfp, samples_folder_name) for sfp in study_folder_path]
print(str(info_file_name_path[0])[-3:])
assert np.array([i.exists() for i in info_file_name_path]).all(), f'Info file does not exist. Check info_file_name_path ({info_file_name_path})'
info_file_name_path

In [3]:
dict_infos_read = {i:h5py_funcs.inspections.read_info_file_to_dict(info_file_name_path=ifp, infoset_name = 'info')  for i,ifp in enumerate(info_file_name_path)}
#dddddd = dict_infos_read.copy()

dict_info_read = {}
#for key in reversed(dict_infos_read.keys()): #reversed to make sure info from initial study_file is contained as a|=b priorizes b
#    dict_info_read = dict_info_read | dddddd[key]

for i,ifp in enumerate(info_file_name_path):
    dict_info_read.update(h5py_funcs.inspections.read_info_file_to_dict(info_file_name_path=ifp, infoset_name = 'info'))

#dict_info_read


#orig:
#dict_info_read = h5py_funcs.inspections.read_info_file_to_dict(info_file_name_path=info_file_name_path, infoset_name = 'info')
#dict_info_read.keys()

##########
#
# The following test fails, because comparing dicts throws an error (below) when arrays are encountered, thats why it is left to the user to make sure the subs are compatible with the main study.
# "The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()" 
#
#########
#if len(info_file_name_path)>1:
#    for i in range(len(info_file_name_path)):
#        for key, value in dict_infos_read[0].items():
#            if key=='time_history': continue
#            else:
#                print(key)
#                if isinstance(value, np.ndarray):
#                    assert (value == dict_infos_read[i][key]).all(), print(value, dict_infos_read[i][key])
#                else:
#                    try: value == dict_infos_read[i][key]
#                    except Exception as e:
#                        print(e)
#                        print(value)
#                        print(dict_infos_read[i][key])

In [ ]:
#print(dict_infos_read[0]['time_history'])
for i in range(len(dict_infos_read)):
    _tmp = []
    for key, value in dict_infos_read[i]['time_history'].items():
        if key == 'attrs':
            continue
        else:
            if not value['attrs']['finished']:
                _tmp.append(key)
    print(len(_tmp), _tmp)
del _tmp

In [ ]:
list_studies = list(dict_info_read['time_history'].keys())
list_studies.remove('attrs')
print(len(list_studies), list_studies)

In [6]:
#print(dict_info_read['time_history']['zz_7293366434']['data'][2].decode('utf-8')[:4] == '2024')
#print(dict_info_read['time_history']['zz_7293366434']['attrs']['finished'])

In [ ]:
for stud in list_studies:
    if (not dict_info_read['time_history'][stud]['attrs']['finished'])\
        and (dict_info_read['time_history'][stud]['data'][2].decode('utf-8')[:4] == '2024'):
        print('here 1', stud)
    elif dict_info_read['time_history'][stud]['attrs']['finished']\
        and (dict_info_read['time_history'][stud]['data'][2].decode('utf-8')[:4] != '2024'):
        print('here 1_2', stud)
    elif dict_info_read['time_history'][stud]['attrs']['finished']\
        and (dict_info_read['time_history'][stud]['data'][2].decode('utf-8')[:4] == '2024'):
        #print('here 2')
        continue
    else:
        print('here else', stud)
        for key, value in dict_infos_read.items():
            if '2024' == value['time_history'][stud]['data'][2].decode('utf-8')[:4]:
                dict_info_read['time_history'][stud]['data'][1] = (value['time_history'][stud]['data'][1]+'.'.encode('utf-8'))[:-1]
                dict_info_read['time_history'][stud]['data'][2] = (value['time_history'][stud]['data'][2]+'.'.encode('utf-8'))[:-1]
                dict_info_read['time_history'][stud]['attrs']['finished'] = True


In [ ]:
for stud in list_studies:
    if not dict_info_read['time_history'][stud]['attrs']['finished']:
        print(stud, dict_info_read['time_history'][stud]['data'])

In [ ]:
print(dict_info_read['study'])
study_read = dict_info_read['study']['data']
study_read.dtype.names
est_runts = study_read['sets']['estimated_runtime']
cum_est_runts = np.sum(est_runts)
print(est_runts)
print('cummulative estimated runtime for 1000 samples each is [h]:', cum_est_runts/3600)
del study_read, est_runts, cum_est_runts


In [ ]:
array_identifiers, started_psets, finished_psets = h5py_funcs.inspections.extract_identifiers(dict_info_read = dict_info_read)

print(started_psets)
print(finished_psets)
print('{p1}/{p2} p_sets have been started, \n {p3}/{p2} p_sets have been finished'.format(p1=len(started_psets), p2=array_identifiers.shape[0], p3=len(finished_psets)))

list_array_identifiers = []; list_started_psets = []; list_finished_psets = []
appenders = [list_array_identifiers, list_started_psets, list_finished_psets]
for d in dict_infos_read.values():
    appendees = h5py_funcs.inspections.extract_identifiers(dict_info_read = d)
    print(len(appendees[0]), len(appendees[1]), len(appendees[2]))
    assert (array_identifiers==appendees[0]).all()
    list_array_identifiers.append(appendees[0])
    if len(list_started_psets) > 0:
        print('here if')
        list_started_psets.append([ap for ap in appendees[1] if not ap in list_started_psets[0]])
        list_finished_psets.append([ap for ap in appendees[2] if not ap in list_finished_psets[0]])
    else:
        print('here else')
        list_started_psets.append(appendees[1])
        list_finished_psets.append(appendees[2])

print(list_started_psets)
print(list_finished_psets)
print('{p1}/{p2} p_sets have been started, \n {p3}/{p2} p_sets have been finished'.\
      format(p1=[len(l) for l in list_started_psets], p2=[len(l) for l in list_array_identifiers], p3=[len(l) for l in list_finished_psets]))

In [11]:
is_read_from_sample_data = False

In [12]:
dict_for_df = {}
if is_read_from_sample_data:
    import os
    for sfnp, ids in zip(samples_folder_name_path,list_finished_psets):
        print('reading from', sfnp)
        #print(os.path.exists(sfnp))
        #for id in ids:
        #    _tmp = os.path.join(sfnp, id.decode('utf-8')+'.h5')
        #    print(os.path.exists(_tmp), _tmp)
        dict_for_df |= h5py_funcs.inspections.read_answers_to_dict(\
            samples_folder_name_path=sfnp\
            , array_identifiers=np.array(ids))#[::25]
    print(dict_for_df.__sizeof__()) # size in memory in bytes
    print(len(dict_for_df)) # number of keys in dict
    with open(f'01_out\\dict_for_df_{study_name}.txt', 'wb') as f:
        pickle.dump(dict_for_df, f, protocol=5)
    del dict_for_df


In [13]:
if not is_read_from_sample_data:
    with open(f'01_out\\dict_for_df_{study_name}.txt', 'rb') as f:
        dict_for_df = pickle.load(f)

In [14]:
is_combine_pickled_dicts = False
if is_combine_pickled_dicts:
    in_dicts = ['01_out\\dict_for_df_sub_5_3.txt'\
               ,'01_out\\dict_for_df_sub_5_3_2.txt']
    out_dicts = ['01_out\\dict_for_df_sub_5_3_combined.txt']
    out_dict = {}
    for in_d in in_dicts:
        print(f'read {in_d}')
        with open(in_d, 'rb') as f:
            out_dict |= pickle.load(f)
    print('finished reading all in_dicts')
    for key, val in out_dict.items():
        sample_set_length = (len(val['custom']['sampleset'].keys()))
        if sample_set_length != 10:
            print(key, sample_set_length)

In [15]:
if is_combine_pickled_dicts:
    out_dicts = ['01_out\\dict_for_df_sub_5_3_combined.txt']
    with open(out_dicts[0], 'wb') as f:
        pickle.dump(out_dict, f, protocol=5)

In [16]:
import ast
import dwave.samplers

num_particles = 5
qubo = dict_info_read['qubos'][f'{num_particles}_{num_particles}']

#sim_annealing_sample = dimod.samplers.ExactSolver().sample_qubo(
#    {ast.literal_eval(key): value['data'] for key, value in qubo.items()})
sim_annealing_sample = dwave.samplers.SimulatedAnnealingSampler().sample_qubo(
    {ast.literal_eval(key): value['data'] for key, value in qubo.items()},
    num_reads=10000)
sim_annealing_sample = sim_annealing_sample.aggregate() # accumulates number of occurences

In [ ]:
type(sim_annealing_sample)

In [ ]:
exact_sol = sim_annealing_sample.record
exact_sol.sort(order='energy')
exact_sol

In [41]:
n_samples_to_compare = 3
n_exact_sols_to_compare = 3
#colour_label = 'fraction_samples_matched_3_samps_3_sols'
colour_label = 'fraction_samples_is_found_best'
dir_name_path_plots = './01_out/'
is_train_gp = False
training_iterations = 5 # only required if is_train_gp==True

In [ ]:
success_dict = h5py_funcs.inspections.extract_success_dict(\
    dict_for_df = dict_for_df\
    , exact_sols = exact_sol\
    , n_samples_to_compare = n_samples_to_compare\
    , n_exact_sols_to_compare = n_exact_sols_to_compare)


for key in success_dict.keys():
    print(key, success_dict[key])

In [43]:
#fig = h5py_funcs.inspections.return_bar_plot(\
#    success_dict = success_dict\
#    , n_samples_to_compare = n_samples_to_compare\
#    , n_exact_sols_to_compare = n_exact_sols_to_compare)



In [ ]:
a = np.zeros((dict_info_read['study']['data'].shape[0],))
print(a)
print(dict_info_read['study']['data']['sets'].shape)
print(dict_info_read['study']['data']['sets'].dtype.isalignedstruct)
b = np.lib.recfunctions.rec_append_fields(dict_info_read['study']['data'], 'param 1', a)
print(b)

In [ ]:
print(b.dtype)

In [ ]:
dict_info_read['study']['data']['sets'].copy().view(dtype=np.float64, type=np.ndarray)

In [ ]:
started_sets, study_matched_started_ids = h5py_funcs.inspections.extract_started_sets_from_success_dict(\
    success_dict = success_dict\
    , dict_info_read = dict_info_read)
print(started_sets)
print(study_matched_started_ids)
print(study_matched_started_ids.dtype)

In [ ]:
results_names = list(list(success_dict.values())[0].keys())
to_remove = ['is_found_best_per_run', 'num_subs_per_run', 'num_samples_per_run', 'num_samples_per_sub_per_run', 'num_matched_per_run'\
             , 'num_matched_per_sub_per_run', 'num_samples_matched_per_run', 'num_samples_matched_per_sub_per_run', 'submissions', 'num_samples_is_found_best_per_run']
for s in to_remove:
    results_names.remove(s)
print(results_names)

sampler_array,results_names, ids = h5py_funcs.inspections.generate_sampler_array_for_plots(\
    success_dict = success_dict\
    , results_names = results_names\
    , started_sets = started_sets\
    , study_matched_started_ids = study_matched_started_ids\
    , n_samples_to_compare = n_samples_to_compare\
    , n_exact_sols_to_compare = n_exact_sols_to_compare)

print(sampler_array.shape)
print(results_names)

In [49]:
import torch
import gpytorch

In [50]:
class MultitaskGPModel(gpytorch.models.ExactGP):
                def __init__(self, train_x, train_y, likelihood):
                    super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
                    self.mean_module = gpytorch.means.MultitaskMean(
                        gpytorch.means.ConstantMean(), num_tasks=sampler_array.shape[1]
                    )
                    self.covar_module = gpytorch.kernels.MultitaskKernel(
                        gpytorch.kernels.RBFKernel(), num_tasks=sampler_array.shape[1]
                    )

                def forward(self, x):
                    mean_x = self.mean_module(x)
                    covar_x = self.covar_module(x)
                    return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)

            # Instantiate multi-task likelihood and GP model 
if study_name == 'sub_2':
    train_x = torch.tensor(study_matched_started_ids['sets'].view(np.float64).copy()).to(torch.float)
    test_x = torch.tensor(dict_info_read['study']['data']['sets'].view(np.float64).copy()).to(torch.float)
else:
    _tmp_train_dtype = study_matched_started_ids['sets'].dtype
    _tmp_test_dtype = dict_info_read['study']['data']['sets'].dtype
    _tmp_train_data = study_matched_started_ids['sets'].view((np.float64, len(_tmp_train_dtype.names))).copy()
    _tmp_test_data = dict_info_read['study']['data']['sets'].view((np.float64, len(_tmp_test_dtype.names))).copy()
    train_x = torch.tensor(_tmp_train_data).to(torch.float)
    test_x = torch.tensor(_tmp_test_data).to(torch.float)
    del(_tmp_train_dtype, _tmp_test_dtype, _tmp_train_data, _tmp_test_data)

train_y = torch.tensor(sampler_array).to(torch.float)

num_params_in_gp = train_x.shape[1]
num_params_out_gp = train_y.shape[1]

likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=sampler_array.shape[1])
model = MultitaskGPModel(train_x, train_y, likelihood)

In [ ]:
if not is_train_gp:
    print('is_train_gp =', is_train_gp, ' GP training is skipped')
    mean = None
    pass
else:
    # Switch to training mode
    model.train()
    likelihood.train()

    # Define optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    # Train GP using training data
    print('Start training')
    print(training_iterations)
    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        optimizer.step()
        print('  Finished training iteration %i/%i' % (i + 1, training_iterations), 'loss:', loss.item())
    print('Finished training' + '\n')
    # Switch to evaluation mode, and probe trained GP using testing data
    model.eval()
    likelihood.eval()
    print('Start testing')
    with torch.no_grad():
        observed_model = model(test_x)
        print('  Testing: Finished evaluation')
        observed_pred = likelihood(observed_model)
        print('  Testing: Finished likelihood')
        mean = observed_pred.mean
        lower, upper = observed_pred.confidence_region()
    print('Finished testing' + '\n')

    assert mean.shape[1] == num_params_out_gp

In [ ]:
sampler_array.shape

In [ ]:
import importlib
importlib.reload(h5py_funcs.inspections)

In [54]:
SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 14
kwargs_pltrc = {'font_size':SMALL_SIZE\
                ,'axes_titlesize': SMALL_SIZE\
                ,'axes_labelsize': MEDIUM_SIZE\
                ,'xtick_labelsize': SMALL_SIZE\
                ,'ytick_labelsize': SMALL_SIZE\
                ,'legend_fontsize': SMALL_SIZE\
                ,'figure_titlesize': BIGGER_SIZE\
                ,'marker_scatter': '2'}

In [ ]:
print(results_names)
print(mean.shape) if np.array(mean).all() else print('GP was not trained')
print(colour_label)
print(dir_name_path_plots)
print(sampler_array.shape)
print(study_matched_started_ids.dtype)
print(study_matched_started_ids.ndim)
print(len(study_matched_started_ids['sets'].dtype.names))
print(study_matched_started_ids['sets'].ndim)
fig2, fig_pp1 = h5py_funcs.inspections.return_plots(study_name = study_name\
        , study_matched_started_ids = study_matched_started_ids\
        , gp_mean = mean, results_names = results_names, dict_info_read = dict_info_read\
        , sampler_array = sampler_array, colour_label = colour_label, dir_name_path_plots = dir_name_path_plots\
        , comb_types=['bi'], kwargs_pltrc=kwargs_pltrc)

In [56]:
#fig2 = fig2.sort_values(by='fraction_samples_is_found_best')
#fig2
#fig_pp1[1]

In [57]:
#fig2.iloc[:,0]

In [58]:
plt.close()

In [ ]:

fig_pp1.show()

In [60]:
import SALib as sa
import SALib.analyze.sobol

In [ ]:
names = study_matched_started_ids['sets'].dtype.names[:3]
bounds = [[study_matched_started_ids['sets'][name].min(), study_matched_started_ids['sets'][name].max()] for name in names]
salib_problem = {
    'num_vars': 3,
    'names': names,
    'bounds': bounds
}
salib_problem

In [ ]:
ar = sampler_array[:,-2]
sa.analyze.sobol.analyze(salib_problem, np.pad(ar, (0, 512-len(ar)), 'constant', constant_values=(0, ar.mean())), print_to_console=True)

In [41]:
ar = sampler_array[:,-2]
ar2 = np.pad(ar, (0, 512-len(ar)), 'constant', constant_values=(0, 0))

In [ ]:
sampler_array, names